# SD210 Machine learning - challenge

***ZHANG Bolong, WANG Yuqing, YAN Ming, ZHU Fangda***

We run this code on Google Colab

---


<---------- For more details, please refer to our report ---------->

---
## Part 2: Return Rate Forecast
Build a return forecast template for each product from a shopping cart


## Loading data

In [0]:
from __future__ import division
import os
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

In [0]:
# customers = pd.read_csv("customers.csv")
# products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

## Feature engineering

### Individual Return Features

---

This part runs for a long time, we have run it before, and have saved the results as ***customers60.csv*** and ***products60.csv***

In [0]:
# customers['CReturnTimes'] = 0
# customers['COrderTimes'] = 0

# products['VReturnTimes'] = 0
# products['VOrderTimes'] = 0
# products['PCReturnTimes'] = 0
# products['PCOrderTimes'] = 0
# products['PReturnTimes'] = 0
# products['POrderTimes'] = 0
# products['BReturnTimes'] = 0
# products['BOrderTimes'] = 0

# for i in range(600000):
#     ReturnBin = y_train.loc[i, 'ReturnQuantityBin']

#     Cid = X_train.loc[i, 'CustomerId']
#     if Cid in customers['CustomerId'].values:
#         customers.loc[customers['CustomerId'] == Cid,
#                       'CReturnTimes'] += ReturnBin
#         customers.loc[customers['CustomerId'] == Cid, 'COrderTimes'] += 1

#     Vid = X_train.loc[i, 'VariantId']
#     if Vid in products['VariantId'].values:
#         products.loc[products['VariantId'] == Vid, 'VReturnTimes'] += ReturnBin
#         products.loc[products['VariantId'] == Vid, 'VOrderTimes'] += 1

#         PCid = products.loc[products['VariantId'] == Vid,
#                             'ProductColorId'].values[0]
#         products.loc[products['ProductColorId'] == PCid,
#                      'PCReturnTimes'] += ReturnBin
#         products.loc[products['ProductColorId'] == PCid, 'PCOrderTimes'] += 1

#         Pid = products.loc[products['VariantId'] == Vid, 'ProductId'].values[0]
#         products.loc[products['ProductId'] == Pid, 'PReturnTimes'] += ReturnBin
#         products.loc[products['ProductId'] == Pid, 'POrderTimes'] += 1

#         Bid = products.loc[products['VariantId'] == Vid, 'BrandId'].values[0]
#         products.loc[products['BrandId'] == Bid, 'BReturnTimes'] += ReturnBin
#         products.loc[products['BrandId'] == Bid, 'BOrderTimes'] += 1

# customers['CReturnRate'] = customers['CReturnTimes'] / \
#                           (customers['COrderTimes'] + 0.1)
# products['VReturnRate'] = products['VReturnTimes'] / \
#                           (products['VOrderTimes'] + 0.1)
# products['PCReturnRate'] = products['PCReturnTimes'] / \
#                           (products['PCOrderTimes'] + 0.1)
# products['PReturnRate'] = products['PReturnTimes'] / \
#                           (products['POrderTimes'] + 0.1)
# products['BReturnRate'] = products['BReturnTimes'] / \
#                           (products['BOrderTimes'] + 0.1)

# customers.to_csv('customers60.csv', index=False)
# products.to_csv('products60.csv', index=False)

In [0]:
customers = pd.read_csv("customers60.csv")
products = pd.read_csv("products60.csv")

### Customer Features

In [5]:
customers[customers['BirthDate'] == "4958-02-12 00:00:00"]

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,CReturnTimes,COrderTimes,CReturnRate
578916,12568532.0,FR,4958-02-12 00:00:00,Femme,2011-12-09 15:19:37,0.0,1,0.0


In [0]:
customers.loc[578916, 'BirthDate'] = "1958-02-12 00:00:00"

customers['IsMale'] = np.where(customers['Gender'] == 'Homme', 1, 0)

customers['FirstOrderTime'] = (
    pd.to_datetime(customers['FirstOrderDate']) -
    pd.to_datetime("2005-09-01 00:00:00")).astype('timedelta64[M]')

customers['FirstOrderHour'] = pd.to_datetime(
    customers['FirstOrderDate']).dt.hour
customers['FirstOrderDay'] = pd.to_datetime(
    customers['FirstOrderDate']).dt.day
customers['FirstOrderWeekday'] = pd.to_datetime(
    customers['FirstOrderDate']).dt.weekday
customers['FirstOrderMonth'] = pd.to_datetime(
    customers['FirstOrderDate']).dt.month

### Product Features

In [0]:
products.rename(columns={'SeasonLabel': 'ProductSeasonLabel'}, inplace=True)

In [0]:
products['IsProductSeasonAH'] = np.where(
    products['ProductSeasonLabel'] == 'Automne/Hiver', 1, 0)

products['IsRemovableSole'] = np.where(products['RemovableSole'], 1, 0)

products['ProductGender'] = products['GenderLabel']
products.loc[(products['GenderLabel'] == 'Sacs') |
             (products['GenderLabel'] == 'Accessoires'),
             'ProductGender'] = 'Not shoes'

In [0]:
products['BrandUniverse'] = products['UniverseLabel']
products.loc[(products['UniverseLabel'] != 'DÃ©tente') &
             (products['UniverseLabel'] != 'Ville') &
             (products['UniverseLabel'] != 'ND'),
             'BrandUniverse'] = 'Other'

products['BrandType'] = products['TypeBrand']
products.loc[(products['TypeBrand'] == 'Super Luxe'), 'BrandType'] = 'Luxe'

products['TypeProduct'] = products['ProductType']
products.loc[(products['ProductType'] != 'Baskets') &
             (products['ProductType'] != 'Sandales et nu-pieds') &
             (products['ProductType'] != 'Bottines et boots') &
             (products['ProductType'] != 'Ballerines') &
             (products['ProductType'] != 'Escarpins') &
             (products['ProductType'] != 'Chaussures Ã  lacets') &
             (products['ProductType'] != 'Chaussures de sport') &
             (products['ProductType'] != 'Bottes') &
             (products['ProductType'] != 'Tongs') &
             (products['ProductType'] != 'Mocassins'),
             'TypeProduct'] = 'Other'

products['IsBlack'] = np.where((products['SupplierColor'] == 'Black') |
                               (products['SupplierColor'] == 'Noir') |
                               (products['SupplierColor'] == 'Black leather') |
                               (products['SupplierColor'] == 'Negro') |
                               (products['SupplierColor'] == 'Anthracite') |
                               (products['SupplierColor'] == 'Nero') |
                               (products['SupplierColor'] == 'Schwarz') |
                               (products['SupplierColor'] == 'Black Suede') |
                               (products['SupplierColor'] == 'Noir (8)') |
                               (products['SupplierColor'] == 'Nero black') |
                               (products['SupplierColor'] == 'Vernis noir') |
                               (products['SupplierColor'] == 'Cuir noir') |
                               (products['SupplierColor'] == 'Black Patent') |
                               (products['SupplierColor'] == 'coal'),
                               1, 0)

products['Subtype'] = products['SubtypeLabel']
products.loc[(products['SubtypeLabel'] != 'Bout rond') &
             (products['SubtypeLabel'] != 'Basses') &
             (products['SubtypeLabel'] != np.nan),
             'Subtype'] = 'Other'

products.loc[(products['SizeAdviceDescription'] ==
              'Prenez votre pointure habituelle') |
             (products['SizeAdviceDescription'] ==
              'Ce modÃ¨le chausse normalement'),
             'Description'] = 'size normal'
products.loc[(products['SizeAdviceDescription'] ==
              'Prenez une taille en dessous de votre pointure habituelle') |
             (products['SizeAdviceDescription'] ==
              'ModÃ¨le confortable, convient aux pieds larges') |
             (products['SizeAdviceDescription'] ==
              'Prenez une taille en dessous de sa pointure !') |
             (products['SizeAdviceDescription'] ==
              'Chaussant gÃ©nÃ©reux. Si vous Ãªtes habituellement entre deux pointures, nous vous conseillons de choisir une demi taille en dessous de votre pointure habituelle.') |
             (products['SizeAdviceDescription'] ==
              'Chaussant gÃ©nÃ©reux. Nous vous conseillons de commander votre pointure habituelle et d\'ajouter Ã©ventuellement une demi-semelle.') |
             (products['SizeAdviceDescription'] ==
              'Chaussant trÃ¨s confortable, convient aux pieds trÃ¨s larges et/ou trÃ¨s sensibles.') |
             (products['SizeAdviceDescription'] ==
              'Chaussant particuliÃ¨rement gÃ©nÃ©reux. Nous vous conseillons de choisir deux tailles en dessous de votre pointure habituelle.'),
             'Description'] = 'size large'
products.loc[(products['SizeAdviceDescription'] ==
              'Prenez une taille au-dessus de votre pointure habituelle') |
             (products['SizeAdviceDescription'] ==
              'ModÃ¨le Ã©troit, convient aux pieds fins') |
             (products['SizeAdviceDescription'] ==
              'Prenez une taille au-dessus de sa pointure !') |
             (products['SizeAdviceDescription'] ==
              'Chaussant petit. Si vous Ãªtes habituellement entre deux pointures, nous vous conseillons de choisir une demi taille au-dessus de votre pointure habituelle.') |
             (products['SizeAdviceDescription'] ==
              'Chaussant petit. Nous vous conseillons de commander une pointure au-dessus de votre pointure habituelle et d\'ajouter Ã©ventuellement une demi-semelle.') |
             (products['SizeAdviceDescription'] ==
              'Chaussant particuliÃ¨rement petit. Nous vous conseillons de choisir deux pointures au-dessus de votre pointure habituelle.'),
             'Description'] = 'size small'

In [0]:
products['ShoesSizeMax'] = np.where((products['ProductGender'] != 'Not shoes') &
                                    (products['MaxSize'] < 60),
                                    products['MaxSize'], np.nan)

products['ShoesSizeMin'] = np.where((products['ProductGender'] != 'Not shoes') &
                                    (products['MaxSize'] < 60),
                                    products['MinSize'], np.nan)

### Merge

In [0]:
X_trainNew = pd.merge(X_train, customers, how='left', on='CustomerId')
X_trainNew = pd.merge(X_trainNew, products, how='left', on='VariantId')
X_testNew = pd.merge(X_test, customers, how='left', on='CustomerId')
X_testNew = pd.merge(X_testNew, products, how='left', on='VariantId')

### New Train and Test Features

In [0]:
g = lambda x: x.replace(",",".")
X_trainNew.UnitPMPEUR = map(np.float64,(map(g,X_trainNew.UnitPMPEUR)))
X_testNew.UnitPMPEUR = map(np.float64,(map(g,X_testNew.UnitPMPEUR)))

In [0]:
X_trainNew['IsOrderDirect'] = np.where(
    X_trainNew['OrderTypelabel'] == 'DIRECT', 1, 0)
X_testNew['IsOrderDirect'] = np.where(
    X_testNew['OrderTypelabel'] == 'DIRECT', 1, 0)

X_trainNew['IsNouveau'] = np.where(
    X_trainNew['CustomerTypeLabel'] == 'Nouveau', 1, 0)
X_testNew['IsNouveau'] = np.where(
    X_testNew['CustomerTypeLabel'] == 'Nouveau', 1, 0)

X_trainNew['IsSeasonAH'] = np.where(
    X_trainNew['SeasonLabel'] == 'Automne/Hiver', 1, 0)
X_testNew['IsSeasonAH'] = np.where(
    X_testNew['SeasonLabel'] == 'Automne/Hiver', 1, 0)

In [0]:
X_trainNew['PayementMode'] = X_trainNew['PayementModeLabel']
X_trainNew.loc[(X_trainNew['PayementModeLabel'] != 'Carte bancaire') &
               (X_trainNew['PayementModeLabel'] != 'PayPal') &
               (X_trainNew['PayementModeLabel'] != 'Klarna_Invoice') &
               (X_trainNew['PayementModeLabel'] != 'iDeal') &
               (X_trainNew['PayementModeLabel'] != 'BankTransfer_DE') &
               (X_trainNew['PayementModeLabel'] != 'Gratuit'),
               'PayementMode'] = 'Other'
X_testNew['PayementMode'] = X_testNew['PayementModeLabel']
X_testNew.loc[(X_testNew['PayementModeLabel'] != 'Carte bancaire') &
              (X_testNew['PayementModeLabel'] != 'PayPal') &
              (X_testNew['PayementModeLabel'] != 'Klarna_Invoice') &
              (X_testNew['PayementModeLabel'] != 'iDeal') &
              (X_trainNew['PayementModeLabel'] != 'BankTransfer_DE') &
              (X_testNew['PayementModeLabel'] != 'Gratuit'),
              'PayementMode'] = 'Other'

X_trainNew['Country'] = X_trainNew['IsoCode']
X_trainNew.loc[(X_trainNew['IsoCode'] != 'FR') &
               (X_trainNew['IsoCode'] != 'DE') &
               (X_trainNew['IsoCode'] != 'BE') &
               (X_trainNew['IsoCode'] != 'NL') &
               (X_trainNew['IsoCode'] != 'GB') &
               (X_trainNew['IsoCode'] != 'ES') &
               (X_trainNew['IsoCode'] != 'IT'),
               'Country'] = 'Other'
X_testNew['Country'] = X_testNew['IsoCode']
X_testNew.loc[(X_testNew['IsoCode'] != 'FR') &
              (X_testNew['IsoCode'] != 'DE') &
              (X_testNew['IsoCode'] != 'BE') &
              (X_testNew['IsoCode'] != 'NL') &
              (X_testNew['IsoCode'] != 'GB') &
              (X_testNew['IsoCode'] != 'ES') &
              (X_testNew['IsoCode'] != 'IT'),
              'Country'] = 'Other'

X_trainNew['DeviceType'] = X_trainNew['DeviceTypeLabel']
X_trainNew.loc[(X_trainNew['DeviceTypeLabel'] != 'ND') &
               (X_trainNew['DeviceTypeLabel'] != 'Desktop'),
               'DeviceType'] = 'Handheld'
X_testNew['DeviceType'] = X_testNew['DeviceTypeLabel']
X_testNew.loc[(X_testNew['DeviceTypeLabel'] != 'ND') &
              (X_testNew['DeviceTypeLabel'] != 'Desktop'),
              'DeviceType'] = 'Handheld'

X_trainNew['IsDeviceND'] = np.where(
    X_trainNew['DeviceTypeLabel'] == 'ND', 1, 0)
X_testNew['IsDeviceND'] = np.where(
    X_testNew['DeviceTypeLabel'] == 'ND', 1, 0)

In [0]:
X_trainNew['IsProductNull'] = np.where(
    pd.isnull(X_trainNew['GenderLabel']), 1, 0)
X_testNew['IsProductNull'] = np.where(
    pd.isnull(X_testNew['GenderLabel']), 1, 0)

X_trainNew['IsCustomerNull'] = np.where(pd.isnull(X_trainNew['Gender']), 1, 0)
X_testNew['IsCustomerNull'] = np.where(pd.isnull(X_testNew['Gender']), 1, 0)

X_trainNew['IsCalfTurnNull'] = np.where(
    pd.isnull(X_trainNew['CalfTurn']), 1, 0)
X_testNew['IsCalfTurnNull'] = np.where(pd.isnull(X_testNew['CalfTurn']), 1, 0)

X_trainNew['IsUpperHeightNull'] = np.where(
    pd.isnull(X_trainNew['UpperHeight']), 1, 0)
X_testNew['IsUpperHeightNull'] = np.where(
    pd.isnull(X_testNew['UpperHeight']), 1, 0)

X_trainNew['IsHeelHeightNull'] = np.where(
    pd.isnull(X_trainNew['HeelHeight']), 1, 0)
X_testNew['IsHeelHeightNull'] = np.where(
    pd.isnull(X_testNew['HeelHeight']), 1, 0)

X_trainNew['IsSubtypeLabelNull'] = np.where(
    pd.isnull(X_trainNew['SubtypeLabel']), 1, 0)
X_testNew['IsSubtypeLabelNull'] = np.where(
    pd.isnull(X_testNew['SubtypeLabel']), 1, 0)

X_trainNew['IsRemovableSoleNull'] = np.where(
    pd.isnull(X_trainNew['RemovableSole']), 1, 0)
X_testNew['IsRemovableSoleNull'] = np.where(
    pd.isnull(X_testNew['RemovableSole']), 1, 0)

X_trainNew['IsDescriptionNull'] = np.where(
    pd.isnull(X_trainNew['SizeAdviceDescription']), 1, 0)
X_testNew['IsDescriptionNull'] = np.where(
    pd.isnull(X_testNew['SizeAdviceDescription']), 1, 0)

X_trainNew['IsMaterialNull'] = np.where(pd.isnull(X_trainNew['UpperMaterialLabel']) &
                                        pd.isnull(X_trainNew['LiningMaterialLabel']) &
                                        pd.isnull(X_trainNew['OutSoleMaterialLabel']), 1, 0)
X_testNew['IsMaterialNull'] = np.where(pd.isnull(X_testNew['UpperMaterialLabel']) &
                                       pd.isnull(X_testNew['LiningMaterialLabel']) &
                                       pd.isnull(X_testNew['OutSoleMaterialLabel']), 1, 0)

In [0]:
X_trainNew['PriceHTRate'] = (
    X_trainNew['PurchasePriceHT'] + 0.1) / (X_trainNew['UnitPMPEUR'] + 0.1)
X_testNew['PriceHTRate'] = (
    X_testNew['PurchasePriceHT'] + 0.1) / (X_testNew['UnitPMPEUR'] + 0.1)

X_trainNew['IsRateHigh'] = np.where(X_trainNew['PriceHTRate'] > 1, 1, 0)
X_trainNew.loc[pd.isnull(X_trainNew['PurchasePriceHT']), 'IsRateHigh'] = np.nan
X_testNew['IsRateHigh'] = np.where(X_testNew['PriceHTRate'] > 1, 1, 0)
X_testNew.loc[pd.isnull(X_testNew['PurchasePriceHT']), 'IsRateHigh'] = np.nan

X_trainNew['OrderInterval'] = (
    pd.to_datetime(X_trainNew['OrderCreationDate']) -
    pd.to_datetime(X_trainNew['FirstOrderDate'])).astype('timedelta64[D]')
X_testNew['OrderInterval'] = (
    pd.to_datetime(X_testNew['OrderCreationDate']) -
    pd.to_datetime(X_testNew['FirstOrderDate'])).astype('timedelta64[D]')

X_trainNew['OrderCreationAge'] = (
    pd.to_datetime(X_trainNew['OrderCreationDate']) -
    pd.to_datetime(X_trainNew['BirthDate'])).astype('timedelta64[Y]')
X_testNew['OrderCreationAge'] = (
    pd.to_datetime(X_testNew['OrderCreationDate']) -
    pd.to_datetime(X_testNew['BirthDate'])).astype('timedelta64[Y]')

In [0]:
X_trainNew['OrderMonth'] = pd.to_datetime(X_trainNew['OrderCreationDate']).dt.month
X_testNew['OrderMonth'] = pd.to_datetime(X_testNew['OrderCreationDate']).dt.month

X_trainNew['OrderDay'] = pd.to_datetime(X_trainNew['OrderCreationDate']).dt.day
X_testNew['OrderDay'] = pd.to_datetime(X_testNew['OrderCreationDate']).dt.day

X_trainNew['OrderHour'] = pd.to_datetime(X_trainNew['OrderCreationDate']).dt.hour
X_testNew['OrderHour'] = pd.to_datetime(X_testNew['OrderCreationDate']).dt.hour

X_trainNew['OrderWeekday'] = pd.to_datetime(X_trainNew['OrderCreationDate']).dt.weekday
X_testNew['OrderWeekday'] = pd.to_datetime(X_testNew['OrderCreationDate']).dt.weekday

In [0]:
X_trainNew.loc[(X_trainNew['OrderMonth'] > 2) & (
    X_trainNew['OrderMonth'] < 6), 'OrderSeason'] = 'spring'
X_trainNew.loc[(X_trainNew['OrderMonth'] > 5) & (
    X_trainNew['OrderMonth'] < 9), 'OrderSeason'] = 'summer'
X_trainNew.loc[(X_trainNew['OrderMonth'] > 8) & (
    X_trainNew['OrderMonth'] < 12), 'OrderSeason'] = 'autumn'
X_trainNew.loc[(X_trainNew['OrderMonth'] > 11) | (
    X_trainNew['OrderMonth'] < 3), 'OrderSeason'] = 'winter'
X_testNew.loc[(X_testNew['OrderMonth'] > 2) & (
    X_testNew['OrderMonth'] < 6), 'OrderSeason'] = 'spring'
X_testNew.loc[(X_testNew['OrderMonth'] > 5) & (
    X_testNew['OrderMonth'] < 9), 'OrderSeason'] = 'summer'
X_testNew.loc[(X_testNew['OrderMonth'] > 8) & (
    X_testNew['OrderMonth'] < 12), 'OrderSeason'] = 'autumn'
X_testNew.loc[(X_testNew['OrderMonth'] > 11) | (
    X_testNew['OrderMonth'] < 3), 'OrderSeason'] = 'winter'

X_trainNew.loc[X_trainNew['OrderDay'] < 11, 'OrderPeriod'] = 'first'
X_trainNew.loc[(X_trainNew['OrderDay'] > 10) & (
    X_trainNew['OrderDay'] < 21), 'OrderPeriod'] = 'middle'
X_trainNew.loc[X_trainNew['OrderDay'] > 20, 'OrderPeriod'] = 'last'
X_testNew.loc[X_testNew['OrderDay'] < 11, 'OrderPeriod'] = 'first'
X_testNew.loc[(X_testNew['OrderDay'] > 10) & (
    X_testNew['OrderDay'] < 21), 'OrderPeriod'] = 'middle'
X_testNew.loc[X_testNew['OrderDay'] > 20, 'OrderPeriod'] = 'last'

X_trainNew.loc[(X_trainNew['OrderHour'] < 7), 'OrderTime'] = 'night'
X_trainNew.loc[(X_trainNew['OrderHour'] > 6) & (
    X_trainNew['OrderHour'] < 13), 'OrderTime'] = 'morning'
X_trainNew.loc[(X_trainNew['OrderHour'] > 12) & (
    X_trainNew['OrderHour'] < 19), 'OrderTime'] = 'afternoon'
X_trainNew.loc[(X_trainNew['OrderHour'] > 18), 'OrderTime'] = 'evening'
X_testNew.loc[(X_testNew['OrderHour'] < 7), 'OrderTime'] = 'night'
X_testNew.loc[(X_testNew['OrderHour'] > 6) & (
    X_testNew['OrderHour'] < 13), 'OrderTime'] = 'morning'
X_testNew.loc[(X_testNew['OrderHour'] > 12) & (
    X_testNew['OrderHour'] < 19), 'OrderTime'] = 'afternoon'
X_testNew.loc[(X_testNew['OrderHour'] > 18), 'OrderTime'] = 'evening'

X_trainNew.loc[X_trainNew['OrderWeekday'] == 0, 'OrderDayofweek'] = 'Monday'
X_trainNew.loc[X_trainNew['OrderWeekday'] == 1, 'OrderDayofweek'] = 'Tuesday'
X_trainNew.loc[X_trainNew['OrderWeekday'] == 2, 'OrderDayofweek'] = 'Wednesday'
X_trainNew.loc[X_trainNew['OrderWeekday'] == 3, 'OrderDayofweek'] = 'Thursday'
X_trainNew.loc[X_trainNew['OrderWeekday'] == 4, 'OrderDayofweek'] = 'Friday'
X_trainNew.loc[X_trainNew['OrderWeekday'] == 5, 'OrderDayofweek'] = 'Saturday'
X_trainNew.loc[X_trainNew['OrderWeekday'] == 6, 'OrderDayofweek'] = 'Sunday'
X_testNew.loc[X_testNew['OrderWeekday'] == 0, 'OrderDayofweek'] = 'Monday'
X_testNew.loc[X_testNew['OrderWeekday'] == 1, 'OrderDayofweek'] = 'Tuesday'
X_testNew.loc[X_testNew['OrderWeekday'] == 2, 'OrderDayofweek'] = 'Wednesday'
X_testNew.loc[X_testNew['OrderWeekday'] == 3, 'OrderDayofweek'] = 'Thursday'
X_testNew.loc[X_testNew['OrderWeekday'] == 4, 'OrderDayofweek'] = 'Friday'
X_testNew.loc[X_testNew['OrderWeekday'] == 5, 'OrderDayofweek'] = 'Saturday'
X_testNew.loc[X_testNew['OrderWeekday'] == 6, 'OrderDayofweek'] = 'Sunday'

X_trainNew['IsOrderWeekend'] = np.where(X_trainNew['OrderWeekday'] > 4, 1, 0)
X_testNew['IsOrderWeekend'] = np.where(X_testNew['OrderWeekday'] > 4, 1, 0)

## Feature transformation

In [0]:
def funk_mask(d):
    " Defining a simple mask over the input data "
    columns_ext = ["OrderNumber", "BillingPostalCode",
                   "OrderCreationDate", "OrderShipDate", "BirthDate", "FirstOrderDate",
                   "VariantId", "CustomerId", "ProductColorId", "ProductId", "BrandId",
                   "OrderStatusLabel", "Quantity", "MarketTargetLabel",

                   "PayementModeLabel", "IsoCode", "DeviceTypeLabel", "IsDeviceND",
                   "GenderLabel", "UniverseLabel", "TypeBrand", "ProductType",
                   "SubtypeLabel", "SizeAdviceDescription", "SupplierColor",
                   "MinSize", "MaxSize",

                   "CalfTurn", "UpperHeight", "CountryISOCode",
                   "UpperMaterialLabel", "LiningMaterialLabel", "OutSoleMaterialLabel",

                   "OrderTypelabel", "CustomerTypeLabel", "SeasonLabel",
                   "Gender", "ProductSeasonLabel", "RemovableSole",

                   "OrderSeason", "OrderPeriod", "OrderDayofweek", "OrderTime",
                   "IsOrderWeekend", "IsRateHigh",
                   ]
    X1 = d.loc[:, [xx for xx in d.columns if xx not in columns_ext]]
    columns2bin = [x for x in X1.columns if X1[x].dtype == np.dtype('O')]
    X2 = pd.get_dummies(X1.loc[:, columns2bin])
    X1 = X1.loc[:, [xx for xx in X1.columns if xx not in columns2bin]]
    res = pd.concat([X1, X2], axis=1)
    res = res.fillna(-99)
    return(res)

In [0]:
x1 = funk_mask(X_trainNew)
x2 = funk_mask(X_testNew)
seleckt_columns = np.intersect1d(x1.columns, x2.columns)
x1 = x1.loc[:, seleckt_columns]
x2 = x2.loc[:, seleckt_columns]

## Supervised learning : XGBoost or LightGBM

In [0]:
#clf = XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.1, colsample_bytree=0.5)
clf = LGBMClassifier(boosting_type='gbdt', n_estimators=100, max_depth=7, learning_rate=0.1,
                     subsample=1, colsample_bytree=0.7, objective='binary')

clf.fit(x1.iloc[600000:1060000], y_train.ReturnQuantityBin[600000:1060000])
y_tosubmit = clf.predict_proba(x2.loc[:, x1.columns])

## Prediction

In [22]:
yres = clf.predict_proba(x1.loc[1060001:, x1.columns])
roc_auc_score(y_train.ReturnQuantityBin.iloc[1060001:], yres[:, 1])

0.716070420445124

In [0]:
# Submission to the system
np.savetxt('y_pred.txt', y_tosubmit[:, 1], fmt='%f')

## Feature importances

In [24]:
importances = list(zip(clf.feature_importances_, x1.columns))
importances.sort(reverse=True)
importances

[(226, 'UnitPMPEUR'),
 (225, 'TotalLineItems'),
 (146, 'CReturnRate'),
 (132, 'OrderNumCustomer'),
 (131, 'COrderTimes'),
 (117, 'OrderCreationAge'),
 (113, 'PReturnRate'),
 (112, 'BReturnRate'),
 (70, 'HeelHeight'),
 (60, 'PriceHTRate'),
 (58, 'OrderHour'),
 (57, 'BOrderTimes'),
 (55, 'CReturnTimes'),
 (53, 'FirstOrderTime'),
 (52, 'POrderTimes'),
 (52, 'BReturnTimes'),
 (49, 'OrderInterval'),
 (49, 'Country_NL'),
 (48, 'PurchasePriceHT'),
 (47, 'IsOnSale'),
 (45, 'OrderMonth'),
 (44, 'ShoesSizeMin'),
 (42, 'Country_DE'),
 (40, 'PayementMode_Gratuit'),
 (40, 'OrderDay'),
 (38, 'Country_FR'),
 (36, 'Country_IT'),
 (33, 'LineItem'),
 (32, 'IsMale'),
 (32, 'FirstOrderHour'),
 (31, 'PCReturnRate'),
 (31, 'Country_GB'),
 (30, 'PReturnTimes'),
 (30, 'FirstOrderDay'),
 (29, 'ShoesSizeMax'),
 (27, 'Description_size small'),
 (25, 'OrderWeekday'),
 (23, 'TypeProduct_Escarpins'),
 (23, 'PCReturnTimes'),
 (23, 'PCOrderTimes'),
 (21, 'TypeProduct_Other'),
 (21, 'PricingTypeLabel_Promo Avec CP'),
